# Import modules

In [1]:
import os
import pandas as pd
import numpy as np

import re
import matplotlib.pyplot as plt

from tqdm import tqdm

import urllib.request as urllib2
import time
from bs4 import BeautifulSoup, SoupStrainer



# Mount google drive

In [2]:
# References:
# [1] https://towardsdatascience.com/different-ways-to-connect-google-drive-to-a-google-colab-notebook-pt-1-de03433d2f7a
# [2] https://stackoverflow.com/questions/54351852/accessing-shared-with-me-with-colab
# [3] https://stackoverflow.com/questions/53581278/test-if-notebook-is-running-on-google-colab

try:
    from google.colab import drive
    from google.colab import files
    # import torch
    # import torch.nn as nn
    # import torch.nn.functional as F
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:

    # mount google drive 
    drive.mount('/content/gdrive/', force_remount=True)

    import os
    #change directory
    try:
        os.chdir('/content/gdrive/MyDrive/MCED/ADC')

    except:
        os.chdir('/content/gdrive/MyDrive/ADC')

In [3]:
# working directory
wdir = os.getcwd()

DF_MSTheses_dir = wdir +'/DF_MSTHESES/'
DF_MSTheses_dir

'h:\\My Drive\\MCED\\ADC/DF_MSTHESES/'

In [4]:

dict_unis = {
      'UPorto'    : 'rabertoup',          # Repositório Aberto da Universidade do Porto (34682)
      'UCoimbra'  : 'estudogl',           # Estudo Geral - Universidade de Coimbra (20368)
      'ULisboa'   : 'ul',                 # Repositório da Universidade de Lisboa (17436)
      'UNLisboa'  : 'run',                # Repositório Institucional da UNL (14415)
      'UMinho'    : 'rum',                # RepositóriUM - Universidade do Minho (13845)
      'UAveiro'   : 'ria',                # RIA - Repositório Institucional da Universidade de Aveiro (13254)
      'UTL'       : 'rutl',               # Repositório da UTL (12863)
      'UCP'       : 'veritati',           # Veritati - Repositório Institucional da Universidade Católica Portuguesa (9289)
      'ISCTE'     : 'iscte',              # Repositório ISCTE (8605)
      'IPPorto'   : 'recipp',             # Repositório Científico do Instituto Politécnico do Porto (7858)
      'UBI'       : 'ubibliorum',         # uBibliorum (6998)
      'UÉvora'    : 'uevora',             # Repositório Científico da Universidade de Évora (5980)
      'UTAD'      : 'utad',               # Repositório da UTAD (5525)
      'IPLisboa'  : 'ripl',               # Repositório Científico do Instituto Politécnico de Lisboa (5232)
      'Lusófona'  : 'recil',              # ReCiL - Repositório Científico Lusófona (5002)
      'UFPessoa'  : 'ufp',                # Repositório Institucional - Universidade Fernando Pessoa (4170)
      'UAlg'      : 'sapientia',          # Sapientia - Universidade do Algarve (3643)
      'ISPA'      : 'ispa',               # Repositório do ISPA - Instituto Universitário (3367)
      'IPCoimbra' : 'ipc',                # Instituto Politécnico de Coimbra (3053)
      'ICOnline'  : 'iconline'            # IC-online (2688)
       }

In [5]:
l_unis = dict_unis.keys()
l_unis

dict_keys(['UPorto', 'UCoimbra', 'ULisboa', 'UNLisboa', 'UMinho', 'UAveiro', 'UTL', 'UCP', 'ISCTE', 'IPPorto', 'UBI', 'UÉvora', 'UTAD', 'IPLisboa', 'Lusófona', 'UFPessoa', 'UAlg', 'ISPA', 'IPCoimbra', 'ICOnline'])

# Auxiliary functions

In [6]:
import os
import pandas as pd

import re
import urllib.request as urllib2
from bs4 import BeautifulSoup, SoupStrainer




def scrap_collection_tree(fullLink, dict_UO, university, l_UO, dir):

    if len(dict_UO) > len(set(dict_UO.values())):
        print('Repeated dict_UO values')

    try:
        df = pd.read_csv(dir +'dict_Collections_' + university +'.csv')
        dict_collections = dict(zip(df.Title, df.UO))
    
    except:

        # opens URL
        response = urllib2.urlopen(fullLink)

        readLink = response.read()

        # parse html file
        soup = BeautifulSoup(readLink,'html.parser') #, from_encoding=response.info().get_param('charset'))

        # list of all links with'/handle/' in the link
        l_handles = soup.find_all(href=re.compile("/handle/"))

        # name of last UO belonging to the UO list
        str_UO =''

        dict_collections = {}

        for l in l_handles:

            # if the link is empty do nothing
            if pd.isna(l.string):
                continue

            # the links we want does not have an image
            if len(re.findall(r'<img(.*?)', str(l))) > 0:
                continue
            
            # if the text belongs to the list of UO, update the UO
            if l.string in dict_UO.keys():
                str_UO = dict_UO[l.string]

            dict_collections[l.string] = str_UO

        
        

    dict_coll = {}
    for key in dict_collections:
        if dict_collections[key] in l_UO.values():
            dict_coll[key] = dict_collections[key]

    df = pd.DataFrame(zip(dict_coll.keys(), dict_coll.values()), columns = ['Title','UO'])
    df.to_csv(dir +'dict_Collections_' + university +'.csv', index= False)


    return dict_collections

# U.Porto

In [7]:
######################################
# Adapt for each univeristy
#
university ='UPorto'
#
#
######################################


# link to repository page
repoLink ='https://repositorio-aberto.up.pt/community-list'

# list of UO to consider
dict_UO = {
       'CIIMAR - Centro Interdisciplinar de Investigação Marinha e Ambiental' :'CIIMAR',
       'FADEUP - Faculdade de Desporto' :'FADEUP',
       'FAUP - Faculdade de Arquitectura' :'FAUP',
       'FBAUP - Faculdade de Belas Artes' :'FBAUP',
       'FCNAUP - Faculdade de Ciências da Nutrição e Alimentação' :'FCNAUP',
       'FCUP - Faculdade de Ciências' :'FCUP',
       'FDUP - Faculdade de Direito' :'FDUP',
       'FEP - Faculdade de Economia' :'FEP',
       'FEUP - Faculdade de Engenharia' :'FEUP',
       'FFUP - Faculdade de Farmácia' :'FFUP',
       'FLUP - Faculdade de Letras' :'FLUP',
       'FMDUP - Faculdade de Medicina Dentária' :'FMDUP',
       'FMUP - Faculdade de Medicina' :'FMUP',
       'FPCEUP - Faculdade de Psicologia e de Ciências da Educação' :'FPCEUP',
       'I3S - Instituto de Investigação e Inovação em Saúde' :'I3S',
       'ICBAS - Instituto de Ciências Biomédicas Abel Salazar' :'ICBAS',
       'ISPUP - Instituto de Saúde Pública' :'ISPUP',
       'PBS - Porto Business School' :'PBS',
       'REIT - Reitoria' :'REIT',
       'Serviços de Ação Social' :'SASUP',
       'Serviços Partilhados' :'SPUP',
        }


l_UO = {
       'CIIMAR - Centro Interdisciplinar de Investigação Marinha e Ambiental' :'CIIMAR',
       'FADEUP - Faculdade de Desporto' :'FADEUP',
       'FAUP - Faculdade de Arquitectura' :'FAUP',
       'FBAUP - Faculdade de Belas Artes' :'FBAUP',
       'FCNAUP - Faculdade de Ciências da Nutrição e Alimentação' :'FCNAUP',
       'FCUP - Faculdade de Ciências' :'FCUP',
       'FDUP - Faculdade de Direito' :'FDUP',
       'FEP - Faculdade de Economia' :'FEP',
       'FEUP - Faculdade de Engenharia' :'FEUP',
       'FFUP - Faculdade de Farmácia' :'FFUP',
       'FLUP - Faculdade de Letras' :'FLUP',
       'FMDUP - Faculdade de Medicina Dentária' :'FMDUP',
       'FMUP - Faculdade de Medicina' :'FMUP',
       'FPCEUP - Faculdade de Psicologia e de Ciências da Educação' :'FPCEUP',
       'I3S - Instituto de Investigação e Inovação em Saúde' :'I3S',
       'ICBAS - Instituto de Ciências Biomédicas Abel Salazar' :'ICBAS',
       'ISPUP - Instituto de Saúde Pública' :'ISPUP',
       'PBS - Porto Business School' :'PBS',
       # 'REIT - Reitoria' :'REIT',
       # 'Serviços de Ação Social' :'SASUP',
       # 'Serviços Partilhados' :'SPUP',
        }

# ["'{}', # {}".format(b_, a_) for a_, b_ in zip(dict_UO.keys(), dict_UO.values())]

# ["{}"".format(a_) for a_ in dict_UO.keys()]


In [8]:
# Dictionary of collections : UO
dict_Collections = scrap_collection_tree(repoLink, dict_UO, university, l_UO, DF_MSTheses_dir)
dict_Collections

{'CIIMAR - Centro Interdisciplinar de Investigação Marinha e Ambiental': 'CIIMAR',
 'CIIMAR - Artigo em Livro de Atas de Conferência Internacional': 'CIIMAR',
 'CIIMAR - Artigo em Livro de Atas de Conferência Nacional': 'CIIMAR',
 'CIIMAR - Artigo em Outras Revistas': 'CIIMAR',
 'CIIMAR - Artigo em Revista Científica Internacional': 'CIIMAR',
 'CIIMAR - Artigo em Revista Científica Nacional': 'CIIMAR',
 'CIIMAR - Capítulo ou Parte de Catálogo': 'CIIMAR',
 'CIIMAR - Capítulo ou Parte de Livro': 'CIIMAR',
 'CIIMAR - Capítulo ou Parte de Relatório': 'CIIMAR',
 'CIIMAR - Catálogo': 'CIIMAR',
 'CIIMAR - Dissertação': 'CIIMAR',
 'CIIMAR - Livro': 'CIIMAR',
 'CIIMAR - Livro de Atas de Conferência Internacional': 'CIIMAR',
 'CIIMAR - Livro de Atas de Conferência Nacional': 'CIIMAR',
 'CIIMAR - Outra Publicação em Livro': 'CIIMAR',
 'CIIMAR - Outra Publicação em Revista Científica Internacional': 'CIIMAR',
 'CIIMAR - Outra Publicação em Revista Científica Nacional': 'CIIMAR',
 'CIIMAR - Outras 

# UCoimbra

In [9]:
######################################
# Adapt for each univeristy
#
university ='UCoimbra'
#
#
#
######################################

# link to hmtl page
repoLink ='https://estudogeral.sib.uc.pt/community-list'


# list of UO to consider
dict_UO = {
            'Colégio das Artes' : 'CA',
            'Faculdade de Ciências do Desporto e Educação Física' : 'FCDEF',
            'Faculdade de Ciências e Tecnologia' : 'FCT',
            'Faculdade de Direito' : 'FD',
            'Faculdade de Economia' : 'FE',
            'Faculdade de Farmácia' : 'FF',
            'Faculdade de Letras' : 'FL',
            'Faculdade de Medicina' : 'FM',
            'Faculdade de Psicologia e Ciências da Educação' : 'FPCE',
            'Instituto de Investigação Interdisciplinar' : 'III',
            'UC - Bibliotecas' : 'UCB',
            'UC - Dissertações e Teses' : 'UCDT',
            'Unidades de Investigação e Desenvolvimento' : 'UID',
}

l_UO = {
        'Colégio das Artes' : 'CA',
        'Faculdade de Ciências do Desporto e Educação Física' : 'FCDEF',
        'Faculdade de Ciências e Tecnologia' : 'FCT',
        'Faculdade de Direito' : 'FD',
        'Faculdade de Economia' : 'FE',
        'Faculdade de Farmácia' : 'FF',
        'Faculdade de Letras' : 'FL',
        'Faculdade de Medicina' : 'FM',
        'Faculdade de Psicologia e Ciências da Educação' : 'FPCE',
        'Instituto de Investigação Interdisciplinar' : 'III',
        # 'UC - Bibliotecas' : 'UCB',
        # 'UC - Dissertações e Teses' : 'UCDT',
        'Unidades de Investigação e Desenvolvimento' : 'UID',
}

In [10]:
# Dictionary of collections : UO
dict_Collections = scrap_collection_tree(repoLink, dict_UO, university, l_UO, DF_MSTheses_dir)


# ULisboa

In [11]:
######################################
# Adapt for each univeristy
#
university ='ULisboa'
#
#
#
######################################


# link to hmtl page
repoLink ='https://repositorio.ul.pt/community-list'


# list of UO to consider
dict_UO = {
           'Bibliotecas da UL (BUL)' :'BUL',
           'Faculdade de Belas-Artes (FBA)' :'FBA',
           'Faculdade de Ciências (FC)' :'FC',
           'Faculdade de Direito (FD)' :'FD',
           'Faculdade de Farmácia (FF)' :'FF',
           'Faculdade de Letras (FL)' :'FL',
           'Faculdade de Medicina Dentária (FMD)' :'FMD',
           'Faculdade de Medicina (FM)' :'FM',
           'Faculdade de Psicologia e Ciências da Educação (FPCE)' :'FPCE',
           'Faculdade de Psicologia (FP)' :'FP',
           'Instituto de Ciências Sociais (ICS)' :'ICS',
           'Instituto de Educação (IE)' :'IE',
           'Instituto de Geografia e Ordenamento do Território (IGOT)' :'IGOT',
           'Instituto de Medicina Molecular João Lobo Antunes (iMM)' :'iMM',
           'Instituto de Saúde Baseada na Evidência (ISBE)' :'ISBE',
           'Museu de Ciência (MC)' :'MC',
           'Museu Nacional de História Natural (MNHN)' :'MNHN',
           'Reitoria (REIT)' :'REIT',
           'Teses & Dissertações' :'T&D',
}

l_UO = {
        # 'Bibliotecas da UL (BUL)' :'BUL',
        'Faculdade de Belas-Artes (FBA)' :'FBA',
        'Faculdade de Ciências (FC)' :'FC',
        'Faculdade de Direito (FD)' :'FD',
        'Faculdade de Farmácia (FF)' :'FF',
        'Faculdade de Letras (FL)' :'FL',
        'Faculdade de Medicina Dentária (FMD)' :'FMD',
        'Faculdade de Medicina (FM)' :'FM',
        'Faculdade de Psicologia e Ciências da Educação (FPCE)' :'FPCE',
        'Faculdade de Psicologia (FP)' :'FP',
        'Instituto de Ciências Sociais (ICS)' :'ICS',
        'Instituto de Educação (IE)' :'IE',
        'Instituto de Geografia e Ordenamento do Território (IGOT)' :'IGOT',
        'Instituto de Medicina Molecular João Lobo Antunes (iMM)' :'iMM',
        'Instituto de Saúde Baseada na Evidência (ISBE)' :'ISBE',
        # 'Museu de Ciência (MC)' :'MC',
        # 'Museu Nacional de História Natural (MNHN)' :'MNHN',
        # 'Reitoria (REIT)' :'REIT',
        # 'Teses & Dissertações' :'T&D',
}

In [12]:
# Dictionary of collections : UO
dict_Collections = scrap_collection_tree(repoLink, dict_UO, university, l_UO, DF_MSTheses_dir)

# UNLisboa

In [13]:
######################################
# Adapt for each univeristy
#
university ='UNLisboa'
#
#
#
######################################

# link to repository page
repoLink ='https://run.unl.pt/community-list'


# list of UO to consider
dict_UO = {
            'Escola Nacional de Saúde Pública (ENSP)' : 'ENSP',
            'Faculdade de Ciências e Tecnologia (FCT)' : 'FCT',
            'Faculdade de Ciências Sociais e Humanas (FCSH)' : 'FCSH',
            'Faculdade de Direito (FD)' : 'FD',
            'Instituto de Higiene e Medicina Tropical (IHMT)' : 'IHMT',
            'Instituto de Tecnologia Química e Biológica (ITQB)' : 'ITQB',
            'NOVA Information Management School (NIMS)' : 'NIMS',
            'NOVA Medical School (NMS)' : 'NMS',
            'Nova School of Business and Economics (NSBE)' : 'NSBE',
            'Pure-RUN' : 'Pure',
}

dict_UO = {
            'Escola Nacional de Saúde Pública (ENSP)' : 'ENSP',
            'Faculdade de Ciências e Tecnologia (FCT)' : 'FCT',
            'Faculdade de Ciências Sociais e Humanas (FCSH)' : 'FCSH',
            'Faculdade de Direito (FD)' : 'FD',
            'Instituto de Higiene e Medicina Tropical (IHMT)' : 'IHMT',
            'Instituto de Tecnologia Química e Biológica (ITQB)' : 'ITQB',
            'NOVA Information Management School (NIMS)' : 'NIMS',
            'NOVA Medical School (NMS)' : 'NMS',
            'Nova School of Business and Economics (NSBE)' : 'NSBE',
            'Pure-RUN' : 'Pure',
}

In [14]:
# Dictionary of collections : UO
dict_Collections = scrap_collection_tree(repoLink, dict_UO, university, l_UO, DF_MSTheses_dir)

# UMinho

In [15]:
######################################
# Adapt for each univeristy
#
university ='UMinho'
#
#
######################################

# link to repository page
repoLink ='https://repositorium.sdum.uminho.pt/community-list'

# list of UO to consider
dict_UO = {
       'Biblioteca da Universidade do Minho | University of Minho Library' :'BUM',
       'Escola de Arquitetura, Arte e Design | School of Architecture, Art and Design' :'EAAD',
       'Escola de Ciências | School of Sciences' :'EC',
       'Escola de Direito | School of Law' :'ED',
       'Escola de Economia e Gestão | School of Economics and Management' :'EEG',
       'Escola de Engenharia | School of Engineering' :'EE',
       'Escola de Letras Artes e Ciências Humanas | School of Arts and Humanities' :'ELACH',
       'Escola de Medicina | School of Medicine' :'EM',
       'Escola de Psicologia | School of Psychology' :'EP',
       'Escola Superior de Enfermagem | School of Nursing' :'ESE',
       'Instituto de Ciências Sociais | Institute of Social Sciences' :'ICS',
       'Instituto de Educação | Institute of Education' :'IE',
       'Instituto de Investigação I3Bs | Research Institute I3Bs' :'I3Bs',
       'Interfaces da Universidade do Minho | University of Minho interfaces' :'IUM',
       'Serviços da Universidade do Minho | University of Minho services' :'SUM',
       'Spin-offs académicas | Academic spin-offs' :'SOA',
       'UMinho Editora |' :'UME',
       'Unidades culturais | University of Minho' :'UC',
}

dict_UO = {
       # 'Biblioteca da Universidade do Minho | University of Minho Library' :'BUM',
       'Escola de Arquitetura, Arte e Design | School of Architecture, Art and Design' :'EAAD',
       'Escola de Ciências | School of Sciences' :'EC',
       'Escola de Direito | School of Law' :'ED',
       'Escola de Economia e Gestão | School of Economics and Management' :'EEG',
       'Escola de Engenharia | School of Engineering' :'EE',
       'Escola de Letras Artes e Ciências Humanas | School of Arts and Humanities' :'ELACH',
       'Escola de Medicina | School of Medicine' :'EM',
       'Escola de Psicologia | School of Psychology' :'EP',
       'Escola Superior de Enfermagem | School of Nursing' :'ESE',
       'Instituto de Ciências Sociais | Institute of Social Sciences' :'ICS',
       'Instituto de Educação | Institute of Education' :'IE',
       'Instituto de Investigação I3Bs | Research Institute I3Bs' :'I3Bs',
       'Interfaces da Universidade do Minho | University of Minho interfaces' :'IUM',
       'Serviços da Universidade do Minho | University of Minho services' :'SUM',
       'Spin-offs académicas | Academic spin-offs' :'SOA',
       # 'UMinho Editora |' :'UME',
       'Unidades culturais | University of Minho' :'UC',
}

In [16]:
# Dictionary of collections : UO
dict_Collections = scrap_collection_tree(repoLink, dict_UO, university, l_UO, DF_MSTheses_dir)

# UAveiro

In [17]:
######################################
# Adapt for each univeristy
#
university ='UAveiro'
#
#
#
######################################

# link to repository page
repoLink ='https://ria.ua.pt/community-list'


# list of UO to consider
dict_UO = {
               'CBC - Centro de Biologia Celular' :'CBC',
               'CESAM - Centro de Estudos do Ambiente e do Mar' :'CESAM',
               'CICECO - Centro de Investigação em Materiais Cerâmicos e Compósitos' :'CICECO',
               'CIDMA - Centro de Investigação e Desenvolvimento em Matemática e Aplicações' :'CIDMA',
               'CIDTFF - Centro de Investigação em Didática e Tecnologia na Formação de Formadores' :'CIDTFF',
               'CINTESIS - Centro de Investigação em Tecnologias e Serviços de Saúde' :'CINTESIS',
               'CIPES - Centro de Investigação de Políticas de Ensino Superior' :'CIPES',
               'CLLC - Centro de Línguas, Literaturas e Culturas' :'CLLC',
               'DAO - Departamento de Ambiente e Ordenamento' :'DAO',
               'DBio - Departamento de Biologia' :'DBio',
               'DCM - Departamento de Ciências Médicas' :'DCM',
               'DCSPT - Departamento de Ciências Sociais, Políticas e do Território' :'DCSPT',
               'DeCA - Departamento de Comunicação e Arte' :'DeCA',
               'DECivil - Departamento de Engenharia Civil' :'DECivil',
               'DEGEIT - Departamento de Economia, Gestão, Engenharia Industrial e Turismo' :'DEGEIT',
               'DEMaC - Departamento de Engenharia de Materiais e Cerâmica' :'DEMaC',
               'DEM - Departamento de Engenharia Mecânica' :'DEM',
               'DEP - Departamento de Educação e Psicologia' :'DEP',
               'DETI - Departamento de Eletrónica, Telecomunicações e Informática' :'DETI',
               'DFis - Departamento de Física' :'DFis',
               'DGeo - Departamento de Geociências' :'DGeo',
               'DigiMedia - Digital Media and Interaction' :'DigiMedia',
               'DLC - Departamento de Línguas e Culturas' :'DLC',
               'DMat - Departamento de Matemática' :'DMat',
               'DQ - Departamento de Química' :'DQ',
               'ESAN - Escola Superior de Design, Gestão e Tecnologia de Produção Aveiro Norte' :'ESAN',
               'ESSUA - Escola Superior de Saúde da Universidade de Aveiro' :'ESSUA',
               'ESTGA - Escola Superior de Tecnologia e Gestão de Águeda' :'ESTGA',
               'GeoBioTec - Geobiociências, Geoengenharias e Geotecnologias' :'GeoBioTec',
               'GOVCOPP - Unidade de Investigação em Governança, Competitividade e Políticas Públicas' :'GOVCOPP',
               'Grupo UA - Ria de Aveiro' :'Grupo UA',
               'I3N-FSCOSD - Institute for Nanostructures, Nanomodelling and Nanofabrication - Physics of Semicondutors, Optoelectronics and Disordered Systems' :'I3N-FSCOSD',
               'IBIMED - Institute of Biomedicine of Aveiro' :'IBIMED',
               'IC-UA - Instituto Confúcio da Universidade de Aveiro' :'IC-UA',
               'ID+: Instituto de Investigação em Design, Media e Cultura' :'ID+',
               'IEETA - Instituto de Engenharia Eletrónica e Engenharia Informática de Aveiro' :'IEETA',
               'INET-md: Instituto de Etnomusicologia - Centro de Estudos em Música e Dança' :'INET-md',
               'ISCA-UA - Instituto Superior de Contabilidade e Administração da Universidade de Aveiro' :'ISCA-UA',
               'IT - Instituto de Telecomunicações' :'IT',
               'Lab3R - Laboratório de Investigação e Reabilitação Respiratória' :'Lab3R',
               'Plataforma Tecnológica da Bicicleta e Mobilidade Suave' :'PTBMS',
               'Plataforma Tecnológica do Mar' :'PTM',
               'QOPNA - Química Orgânica, Produtos Naturais e Agroalimentares' :'QOPNA',
               'Reitoria' :'REIT',
               'REQUIMTE - Laboratório Associado para a Química Verde - Tecnologias e Processos Limpos' :'REQUIMTE',
               'RISCO - Centro de Investigação de Riscos e Sustentabilidade na Construção' :'RISCO',
               'SBIDM - Serviços de Biblioteca, Informação Documental e Museologia' :'SBIDM',
               'TEMA - Centro de Tecnologia Mecânica e Automação' :'TEMA',
               'UA - Dissertações e Teses da Universidade de Aveiro' :'UA',
               'WJCR - Centro de Investigação William James' :'WJCR',
}

l_UO = {
        'CBC - Centro de Biologia Celular' :'CBC',
        'CESAM - Centro de Estudos do Ambiente e do Mar' :'CESAM',
        'CICECO - Centro de Investigação em Materiais Cerâmicos e Compósitos' :'CICECO',
        'CIDMA - Centro de Investigação e Desenvolvimento em Matemática e Aplicações' :'CIDMA',
        'CIDTFF - Centro de Investigação em Didática e Tecnologia na Formação de Formadores' :'CIDTFF',
        'CINTESIS - Centro de Investigação em Tecnologias e Serviços de Saúde' :'CINTESIS',
        'CIPES - Centro de Investigação de Políticas de Ensino Superior' :'CIPES',
        'CLLC - Centro de Línguas, Literaturas e Culturas' :'CLLC',
        'DAO - Departamento de Ambiente e Ordenamento' :'DAO',
        'DBio - Departamento de Biologia' :'DBio',
        'DCM - Departamento de Ciências Médicas' :'DCM',
        'DCSPT - Departamento de Ciências Sociais, Políticas e do Território' :'DCSPT',
        'DeCA - Departamento de Comunicação e Arte' :'DeCA',
        'DECivil - Departamento de Engenharia Civil' :'DECivil',
        'DEGEIT - Departamento de Economia, Gestão, Engenharia Industrial e Turismo' :'DEGEIT',
        'DEMaC - Departamento de Engenharia de Materiais e Cerâmica' :'DEMaC',
        'DEM - Departamento de Engenharia Mecânica' :'DEM',
        'DEP - Departamento de Educação e Psicologia' :'DEP',
        'DETI - Departamento de Eletrónica, Telecomunicações e Informática' :'DETI',
        'DFis - Departamento de Física' :'DFis',
        'DGeo - Departamento de Geociências' :'DGeo',
        'DigiMedia - Digital Media and Interaction' :'DigiMedia',
        'DLC - Departamento de Línguas e Culturas' :'DLC',
        'DMat - Departamento de Matemática' :'DMat',
        'DQ - Departamento de Química' :'DQ',
        'ESAN - Escola Superior de Design, Gestão e Tecnologia de Produção Aveiro Norte' :'ESAN',
        'ESSUA - Escola Superior de Saúde da Universidade de Aveiro' :'ESSUA',
        'ESTGA - Escola Superior de Tecnologia e Gestão de Águeda' :'ESTGA',
        'GeoBioTec - Geobiociências, Geoengenharias e Geotecnologias' :'GeoBioTec',
        'GOVCOPP - Unidade de Investigação em Governança, Competitividade e Políticas Públicas' :'GOVCOPP',
        'Grupo UA - Ria de Aveiro' :'Grupo UA',
        'I3N-FSCOSD - Institute for Nanostructures, Nanomodelling and Nanofabrication - Physics of Semicondutors, Optoelectronics and Disordered Systems' :'I3N-FSCOSD',
        'IBIMED - Institute of Biomedicine of Aveiro' :'IBIMED',
        'IC-UA - Instituto Confúcio da Universidade de Aveiro' :'IC-UA',
        'ID+: Instituto de Investigação em Design, Media e Cultura' :'ID+',
        'IEETA - Instituto de Engenharia Eletrónica e Engenharia Informática de Aveiro' :'IEETA',
        'INET-md: Instituto de Etnomusicologia - Centro de Estudos em Música e Dança' :'INET-md',
        'ISCA-UA - Instituto Superior de Contabilidade e Administração da Universidade de Aveiro' :'ISCA-UA',
        'IT - Instituto de Telecomunicações' :'IT',
        'Lab3R - Laboratório de Investigação e Reabilitação Respiratória' :'Lab3R',
        'Plataforma Tecnológica da Bicicleta e Mobilidade Suave' :'PTBMS',
        'Plataforma Tecnológica do Mar' :'PTM',
        'QOPNA - Química Orgânica, Produtos Naturais e Agroalimentares' :'QOPNA',
        # 'Reitoria' :'REIT',
        'REQUIMTE - Laboratório Associado para a Química Verde - Tecnologias e Processos Limpos' :'REQUIMTE',
        'RISCO - Centro de Investigação de Riscos e Sustentabilidade na Construção' :'RISCO',
        # 'SBIDM - Serviços de Biblioteca, Informação Documental e Museologia' :'SBIDM',
        'TEMA - Centro de Tecnologia Mecânica e Automação' :'TEMA',
        # 'UA - Dissertações e Teses da Universidade de Aveiro' :'UA',
        'WJCR - Centro de Investigação William James' :'WJCR',
}

In [18]:
# Dictionary of collections : UO
dict_Collections = scrap_collection_tree(repoLink, dict_UO, university, l_UO, DF_MSTheses_dir)

# UTL

In [19]:
######################################
# Adapt for each univeristy
#
university ='UTL'
#
#
######################################

# link to repository page
repoLink ='https://www.repository.utl.pt/community-list'

# list of UO to consider
dict_UO = {
       'CIÊNCIAVITAE(em fase de teste)' :'CV',
       'Faculdade de Medicina Veterinária' :'FMV',
       'FA - Faculdade de Arquitectura' :'FA',
       'FMH - Faculdade de Motricidade Humana' :'FMH',
       'ISA - Instituto Superior de Agronomia' :'ISA',
       'ISCSP - Instituto Superior de Ciências Sociais e Políticas' :'ISCSP',
       'ISEG - Instituto Superior de Economia e Gestão / Lisbon School of Economics & Management' :'ISEG',
}

l_UO = {
       'CIÊNCIAVITAE(em fase de teste)' :'CV',
       'Faculdade de Medicina Veterinária' :'FMV',
       'FA - Faculdade de Arquitectura' :'FA',
       'FMH - Faculdade de Motricidade Humana' :'FMH',
       'ISA - Instituto Superior de Agronomia' :'ISA',
       'ISCSP - Instituto Superior de Ciências Sociais e Políticas' :'ISCSP',
       'ISEG - Instituto Superior de Economia e Gestão / Lisbon School of Economics & Management' :'ISEG',
}

In [20]:
# Dictionary of collections : UO
dict_Collections = scrap_collection_tree(repoLink, dict_UO, university, l_UO, DF_MSTheses_dir)

# UCP

In [21]:
######################################
# Adapt for each univeristy
#
university ='UCP'
#
#
#
######################################

# link to repository page
repoLink ='https://repositorio.ucp.pt/community-list'


# list of UO to consider
dict_UO = {
       'Bibliotecas' :'BIB',
       'Católica Lisbon School of Business & Economics' :'CLSBE',
       'Católica Porto Business School' :'CPBS',
       'Centro de Estudos e Sondagens de Opinião' :'CESO',
       'Departamento de Arquitectura, Ciências e Tecnologia' :'DACT',
       'Departamento de Letras' :'DL',
       'Escola das Artes' :'EA',
       'Escola Superior de Biotecnologia' :'ESB',
       'Faculdade de Ciências Humanas' :'FCH',
       'Faculdade de Ciências Sociais' :'FCS',
       'Faculdade de Direito' :'FD',
       'Faculdade de Educação e Psicologia' :'FEP',
       'Faculdade de Engenharia' :'FE',
       'Faculdade de Filosofia e Ciências Sociais' :'FFCS',
       'Faculdade de Medicina' :'FM',
       'Faculdade de Medicina Dentária' :'FMD',
       'Faculdade de Teologia' :'FT',
       'Instituto de Bioética' :'IB',
       'Instituto de Ciências da Saúde' :'ICS',
       'Instituto de Estudos Europeus' :'IEE',
       'Instituto de Estudos Políticos' :'IEP',
       'Instituto de Gestão e das Organizações da Saúde' :'IGOS',
       'Instituto Superior de Direito Canónico' :'ISDC',
       'Reitoria' :'R',
       'Universidade Católica Editora' :'UCE',
}

l_UO = {
       'Bibliotecas' :'BIB',
       'Católica Lisbon School of Business & Economics' :'CLSBE',
       'Católica Porto Business School' :'CPBS',
       'Centro de Estudos e Sondagens de Opinião' :'CESO',
       'Departamento de Arquitectura, Ciências e Tecnologia' :'DACT',
       'Departamento de Letras' :'DL',
       'Escola das Artes' :'EA',
       'Escola Superior de Biotecnologia' :'ESB',
       'Faculdade de Ciências Humanas' :'FCH',
       'Faculdade de Ciências Sociais' :'FCS',
       'Faculdade de Direito' :'FD',
       'Faculdade de Educação e Psicologia' :'FEP',
       'Faculdade de Engenharia' :'FE',
       'Faculdade de Filosofia e Ciências Sociais' :'FFCS',
       'Faculdade de Medicina' :'FM',
       'Faculdade de Medicina Dentária' :'FMD',
       'Faculdade de Teologia' :'FT',
       'Instituto de Bioética' :'IB',
       'Instituto de Ciências da Saúde' :'ICS',
       'Instituto de Estudos Europeus' :'IEE',
       'Instituto de Estudos Políticos' :'IEP',
       'Instituto de Gestão e das Organizações da Saúde' :'IGOS',
       'Instituto Superior de Direito Canónico' :'ISDC',
       # 'Reitoria' :'R',
       # 'Universidade Católica Editora' :'UCE',
}

In [22]:
# Dictionary of collections : UO
dict_Collections = scrap_collection_tree(repoLink, dict_UO, university, l_UO, DF_MSTheses_dir)

# UBI

In [23]:
######################################
# Adapt for each univeristy
#
university ='UBI'
#
#
#
######################################

# link to repository page
repoLink ='https://ubibliorum.ubi.pt/community-list'


# list of UO to consider
dict_UO = {
           'Centros' :'C',
           'Faculdade de Artes e Letras' :'FAL',
           'Faculdade de Ciências' :'FC',
           'Faculdade de Ciências da Saúde' :'FCS',
           'Faculdade de Ciências Sociais e Humanas' :'FCSH',
           'Faculdade Engenharia' :'FE',
           'ICI - Instituto Coordenador da Investigação' :'ICI',
           'Parcerias' :'P',
}

l_UO = {
        'Centros' :'C',
        'Faculdade de Artes e Letras' :'FAL',
        'Faculdade de Ciências' :'FC',
        'Faculdade de Ciências da Saúde' :'FCS',
        'Faculdade de Ciências Sociais e Humanas' :'FCSH',
        'Faculdade Engenharia' :'FE',
        'ICI - Instituto Coordenador da Investigação' :'ICI',
        'Parcerias' :'P',
}

In [24]:
# Dictionary of collections : UO
dict_Collections = scrap_collection_tree(repoLink, dict_UO, university, l_UO, DF_MSTheses_dir)

# UEvora

In [26]:
######################################
# Adapt for each univeristy
#
university ='UEvora'
#
#
######################################

# link to repository page
repoLink ='http://dspace.uevora.pt/rdpc/community-list'

# list of UO to consider
dict_UO = {
              'Biblioteca Geral' : 'BG',
              'Cátedra City University of Macau' : 'CCUM',
              'Cátedra de Estudos Ibéricos' : 'CEI',
              'Cátedra Energias Renováveis' : 'CER',
              'Cátedra LifeSpan' : 'CL',
              'Cátedra UNESCO em Património Imaterial e Saber-Fazer Tradicional' : 'CUPISFT',
              'CEFAGE - Centro de Estudos e Formação Avançada em Gestão e Economia' : 'CEFAGE',
              'CEHFCi - Centro de Estudos de História e Filosofia da Ciência' : 'CEHFC',
              'CEL - Centro de Estudos em Letras' : 'CEL',
              'CEM - Centro de Engenharia Mecatrónica' : 'CEM',
              'CESEM - Centro de Estudos de Sociologia e Estética Musical' : 'CESEM',
              'CGE - Centro de Geofísica de Évora' : 'CGE',
              'CHAIA - Centro de História da Arte e Investigação Artística' : 'CHAIA',
              'CHRC - Comprehensive Health Research Centre' : 'CHRC',
              'CIBIO-UE - Centro de Investigação em Biodiversidade e Recursos Genéticos' : 'CIBIO-UE',
              'CICP - Centro de Investigação em Ciência Política' : 'CICP',
              'CICS.NOVA - Centro Interdisciplinar de Ciências Sociais' : 'CICS.NOVA',
              'CICTS - Centro de Investigação em Ciências e Tecnologias da Saúde' : 'CICTS',
              'CIDEHUS - Centro Interdisciplinar de História, Culturas e Sociedades' : 'CIDEHUS',
              'CIDESD - Centro de Investigação em Desporto, Saúde e Desenvolvimento Humano' : 'CIDESD',
              'CIEP - Centro de Investigação em Educação e Psicologia' : 'CIEP',
              'CIMA - Centro de Investigação em Matemática e Aplicações' : 'CIMA',
              'CISAAS - Centro de Investigação em Sociologia a Antropologia Augusto da Silva' : 'CISAAS',
              'CITIUE - Centro de Investigação em Tecnologias de Informação' : 'CITIUE',
              'CQE - Centro de Química de Évora' : 'CQE',
              'Departamento de Arquitectura' : 'DA',
              'Departamento de Artes Cénicas' : 'DAC',
              'Departamento de Artes Visuais e Design' : 'DAVD',
              'Departamento de Biologia' : 'DB',
              'Departamento de Ciências Médicas e da Saúde' : 'DCMS',
              'Departamento de Desporto e Saúde' : 'DDS',
              'Departamento de Economia' : 'DEco',
              'Departamento de Enfermagem' : 'DEnf',
              'Departamento de Engenharia Mecatrónica' : 'DEM',
              'Departamento de Engenharia Rural' : 'DER',
              'Departamento de Filosofia' : 'DFil',
              'Departamento de Física' : 'DFis',
              'Departamento de Fitotecnia' : 'DFito',
              'Departamento de Geociências' : 'DGeo',
              'Departamento de Gestão' : 'DGes',
              'Departamento de História' : 'DHist',
              'Departamento de Informática' : 'DInf',
              'Departamento de Linguística e Literaturas' : 'DLL',
              'Departamento de Matemática' : 'DMat',
              'Departamento de Medicina Veterinária' : 'DMV',
              'Departamento de Música' : 'DMus',
              'Departamento de Paisagem, Ambiente e Ordenamento' : 'DPAO',
              'Departamento de Pedagogia e Educação' : 'DPE',
              'Departamento de Psicologia' : 'DPsic',
              'Departamento de Química' : 'DQ',
              'Departamento de Sociologia' : 'DSoc',
              'Departamento de Zootecnia' : 'DZoo',
              'GeoBioTec - GeoBioCiências, GeoTecnologias e GeoEngenharias' : 'GeoBioTec',
              'HERCULES - Laboratório HERCULES - Herança Cultural, Estudos e Salvaguarda' : 'HERCULES',
              'ICT - Instituto de Ciências da Terra' : 'ICT',
              'IFP - Instituto de Filosofia Prática' : 'IFP',
              'IHC - Instituto de História Contemporânea' : 'IHC',
              'INBIO - Rede de Investigação em Biodiversidade e Biologia Evolutiva' : 'INBIO',
              'LAVQ-REQUIMTE - Laboratório Associado para a Química Verde - Tecnologias e Processos Limpos' : 'LAVQ',
              'LISP - Laboratório de Informática, Sistemas e Paralelismo' : 'LISP',
              'MARE-UE - Centro de Ciências do Mar e do Ambiente' : 'MARE-UE',
              'MED - Instituto Mediterrâneo para Agricultura, Ambiente e Desenvolvimento' : 'MED',
              'NICPRI.UÉ - Núcleo de Investigação em Ciência Política e Relações Internacionais' : 'NICPRI.UÉ',
              'NOVALINCS - NOVA Laboratory for Computer Science and Informatics' : 'NOVALINCS',
              'PRAXIS - Centro de Filosofia, Política e Cultura' : 'PRAXIS',
              'SIUE' : 'SIUE',
              'UnIMeM - Unidade de Investigação em Música e Musicologia' : 'UnIMeM',
}

l_UO = {
        # 'Biblioteca Geral' : 'BG',
        'Cátedra City University of Macau' : 'CCUM',
        'Cátedra de Estudos Ibéricos' : 'CEI',
        'Cátedra Energias Renováveis' : 'CER',
        'Cátedra LifeSpan' : 'CL',
        'Cátedra UNESCO em Património Imaterial e Saber-Fazer Tradicional' : 'CUPISFT',
        'CEFAGE - Centro de Estudos e Formação Avançada em Gestão e Economia' : 'CEFAGE',
        'CEHFCi - Centro de Estudos de História e Filosofia da Ciência' : 'CEHFC',
        'CEL - Centro de Estudos em Letras' : 'CEL',
        'CEM - Centro de Engenharia Mecatrónica' : 'CEM',
        'CESEM - Centro de Estudos de Sociologia e Estética Musical' : 'CESEM',
        'CGE - Centro de Geofísica de Évora' : 'CGE',
        'CHAIA - Centro de História da Arte e Investigação Artística' : 'CHAIA',
        'CHRC - Comprehensive Health Research Centre' : 'CHRC',
        'CIBIO-UE - Centro de Investigação em Biodiversidade e Recursos Genéticos' : 'CIBIO-UE',
        'CICP - Centro de Investigação em Ciência Política' : 'CICP',
        'CICS.NOVA - Centro Interdisciplinar de Ciências Sociais' : 'CICS.NOVA',
        'CICTS - Centro de Investigação em Ciências e Tecnologias da Saúde' : 'CICTS',
        'CIDEHUS - Centro Interdisciplinar de História, Culturas e Sociedades' : 'CIDEHUS',
        'CIDESD - Centro de Investigação em Desporto, Saúde e Desenvolvimento Humano' : 'CIDESD',
        'CIEP - Centro de Investigação em Educação e Psicologia' : 'CIEP',
        'CIMA - Centro de Investigação em Matemática e Aplicações' : 'CIMA',
        'CISAAS - Centro de Investigação em Sociologia a Antropologia Augusto da Silva' : 'CISAAS',
        'CITIUE - Centro de Investigação em Tecnologias de Informação' : 'CITIUE',
        'CQE - Centro de Química de Évora' : 'CQE',
        'Departamento de Arquitectura' : 'DA',
        'Departamento de Artes Cénicas' : 'DAC',
        'Departamento de Artes Visuais e Design' : 'DAVD',
        'Departamento de Biologia' : 'DB',
        'Departamento de Ciências Médicas e da Saúde' : 'DCMS',
        'Departamento de Desporto e Saúde' : 'DDS',
        'Departamento de Economia' : 'DEco',
        'Departamento de Enfermagem' : 'DEnf',
        'Departamento de Engenharia Mecatrónica' : 'DEM',
        'Departamento de Engenharia Rural' : 'DER',
        'Departamento de Filosofia' : 'DFil',
        'Departamento de Física' : 'DFis',
        'Departamento de Fitotecnia' : 'DFito',
        'Departamento de Geociências' : 'DGeo',
        'Departamento de Gestão' : 'DGes',
        'Departamento de História' : 'DHist',
        'Departamento de Informática' : 'DInf',
        'Departamento de Linguística e Literaturas' : 'DLL',
        'Departamento de Matemática' : 'DMat',
        'Departamento de Medicina Veterinária' : 'DMV',
        'Departamento de Música' : 'DMus',
        'Departamento de Paisagem, Ambiente e Ordenamento' : 'DPAO',
        'Departamento de Pedagogia e Educação' : 'DPE',
        'Departamento de Psicologia' : 'DPsic',
        'Departamento de Química' : 'DQ',
        'Departamento de Sociologia' : 'DSoc',
        'Departamento de Zootecnia' : 'DZoo',
        'GeoBioTec - GeoBioCiências, GeoTecnologias e GeoEngenharias' : 'GeoBioTec',
        'HERCULES - Laboratório HERCULES - Herança Cultural, Estudos e Salvaguarda' : 'HERCULES',
        'ICT - Instituto de Ciências da Terra' : 'ICT',
        'IFP - Instituto de Filosofia Prática' : 'IFP',
        'IHC - Instituto de História Contemporânea' : 'IHC',
        'INBIO - Rede de Investigação em Biodiversidade e Biologia Evolutiva' : 'INBIO',
        'LAVQ-REQUIMTE - Laboratório Associado para a Química Verde - Tecnologias e Processos Limpos' : 'LAVQ',
        'LISP - Laboratório de Informática, Sistemas e Paralelismo' : 'LISP',
        'MARE-UE - Centro de Ciências do Mar e do Ambiente' : 'MARE-UE',
        'MED - Instituto Mediterrâneo para Agricultura, Ambiente e Desenvolvimento' : 'MED',
        'NICPRI.UÉ - Núcleo de Investigação em Ciência Política e Relações Internacionais' : 'NICPRI.UÉ',
        'NOVALINCS - NOVA Laboratory for Computer Science and Informatics' : 'NOVALINCS',
        'PRAXIS - Centro de Filosofia, Política e Cultura' : 'PRAXIS',
        'SIUE' : 'SIUE',
        'UnIMeM - Unidade de Investigação em Música e Musicologia' : 'UnIMeM',
}

In [27]:
# Dictionary of collections : UO
dict_Collections = scrap_collection_tree(repoLink, dict_UO, university, l_UO, DF_MSTheses_dir)

# UTAD

In [28]:
######################################
# Adapt for each univeristy
#
university ='UTAD'
#
#
#
######################################


# link to repository page
repoLink ='https://repositorio.utad.pt/community-list'


# list of UO to consider
dict_UO = {
           'CECAV - Centro de Ciência Animal e Veterinária' :'CECAV',
           'CEISDTAD - Centro de Estudos e Investigação de Segurança e Defesa de Trás-os-Montes e Alto Douro' :'CEISDTAD',
           'CEL - Centro de Estudos em Letras' :'CEL',
           'CETRAD - Centro de Estudos Transdisciplinares para o Desenvolvimento' :'CETRAD',
           'CGB - Centro de Genómica e Biotecnologia' :'CGB',
           'CIDESD - Centro de Investigação de Desporto, Saúde e Desenvolvimento Humano' :'CIDESD',
           'CITAB - Centro de Investigação e de Tecnologias Agroambientais e Biológicas' :'CITAB',
           'CQ - Centro de Química' :'CQ',
           'Depósitos Anteriores a 2015' :'Até2015',
           'ECAV - Escola de Ciências Agrárias e Veterinárias' :'ECAV',
           'ECHS - Escola de Ciências Humanas e Sociais' :'ECHS',
           'ECT - Escola de Ciências e Tecnologia' :'ECT',
           'ECVA - Escola de Ciências da Vida e do Ambiente' :'ECVA',
           'ESEVR - Escola Superior de Enfermagem' :'ESEVR',
           'Polo CMAT-UTAD' :'CMAT',
           'Pós-Doc' :'Pós-Doc',
           'Provas de Agregação' :'Agregações',
           'SEE - Serviços e Estruturas Especializadas' :'SEE',
           'TD - Teses e Dissertações' :'TD',
}


l_UO = {
        'CECAV - Centro de Ciência Animal e Veterinária' :'CECAV',
        'CEISDTAD - Centro de Estudos e Investigação de Segurança e Defesa de Trás-os-Montes e Alto Douro' :'CEISDTAD',
        'CEL - Centro de Estudos em Letras' :'CEL',
        'CETRAD - Centro de Estudos Transdisciplinares para o Desenvolvimento' :'CETRAD',
        'CGB - Centro de Genómica e Biotecnologia' :'CGB',
        'CIDESD - Centro de Investigação de Desporto, Saúde e Desenvolvimento Humano' :'CIDESD',
        'CITAB - Centro de Investigação e de Tecnologias Agroambientais e Biológicas' :'CITAB',
        'CQ - Centro de Química' :'CQ',
        # 'Depósitos Anteriores a 2015' :'Até2015',
        'ECAV - Escola de Ciências Agrárias e Veterinárias' :'ECAV',
        'ECHS - Escola de Ciências Humanas e Sociais' :'ECHS',
        'ECT - Escola de Ciências e Tecnologia' :'ECT',
        'ECVA - Escola de Ciências da Vida e do Ambiente' :'ECVA',
        'ESEVR - Escola Superior de Enfermagem' :'ESEVR',
        # 'Polo CMAT-UTAD' :'CMAT',
        # 'Pós-Doc' :'Pós-Doc',
        # 'Provas de Agregação' :'Agregações',
        # 'SEE - Serviços e Estruturas Especializadas' :'SEE',
        # 'TD - Teses e Dissertações' :'TD',
}

In [29]:
# Dictionary of collections : UO
dict_Collections = scrap_collection_tree(repoLink, dict_UO, university, l_UO, DF_MSTheses_dir)

# UAlg

In [30]:
######################################
# Adapt for each univeristy
#
university ='UAlg'
#
#
######################################


# link to repository page
repoLink ='https://sapientia.ualg.pt/community-list'

# list of UO to consider
dict_UO = {
            'I. Componente Universitária' : 'I-CU',
            'Faculdade de Ciências e Tecnologia' : 'FCT',
            'Faculdade de Ciências Humanas e Sociais' : 'FCHS',
            'Faculdade de Economia' : 'FE',
            'Faculdade de Medicina e Ciências Biomédicas' : 'FMCB',
            'II. Componente Politécnica' : 'II-CP',
            'Escola Superior de Educação e Comunicação' : 'ESEC',
            'Escola Superior de Gestão, Hotelaria e Turismo' : 'ESGHT',
            'Escola Superior de Saúde' : 'ESS',
            'Instituto Superior de Engenharia' : 'ISE',
            'III. Unidades de Investigação, Desenvolvimento e Outras' : 'III-UIDO',
            'BioMed-SWORD' : 'BioMed-SWORD',
            'ABC-RI' : 'ABC-RI',
            'CCMAR' : 'CCMAR',
            'CEAO' : 'CEAO',
            'CEDS' : 'CEDS',
            'CIAC' : 'CIAC',
            'CIMA' : 'CIMA',
            'CinTurs' : 'CinTurs',
            'ICArEHB' : 'ICArEHB',
            'Ualg' : 'Ualg',
            'IV. Entidades Cooperantes' : 'IV-EC',
            'CHUA' : 'CHUA',
            'CVIVA' : 'CVIVA',
            'UNCEH' : 'UNCEH',
}

l_UO = {
        # 'I. Componente Universitária' : 'I-CU',
        'Faculdade de Ciências e Tecnologia' : 'FCT',
        'Faculdade de Ciências Humanas e Sociais' : 'FCHS',
        'Faculdade de Economia' : 'FE',
        'Faculdade de Medicina e Ciências Biomédicas' : 'FMCB',
        # 'II. Componente Politécnica' : 'II-CP',
        'Escola Superior de Educação e Comunicação' : 'ESEC',
        'Escola Superior de Gestão, Hotelaria e Turismo' : 'ESGHT',
        'Escola Superior de Saúde' : 'ESS',
        'Instituto Superior de Engenharia' : 'ISE',
        # 'III. Unidades de Investigação, Desenvolvimento e Outras' : 'III-UIDO',
        'BioMed-SWORD' : 'BioMed-SWORD',
        'ABC-RI' : 'ABC-RI',
        'CCMAR' : 'CCMAR',
        'CEAO' : 'CEAO',
        'CEDS' : 'CEDS',
        'CIAC' : 'CIAC',
        'CIMA' : 'CIMA',
        'CinTurs' : 'CinTurs',
        'ICArEHB' : 'ICArEHB',
        'Ualg' : 'Ualg',
        # 'IV. Entidades Cooperantes' : 'IV-EC',
        'CHUA' : 'CHUA',
        'CVIVA' : 'CVIVA',
        'UNCEH' : 'UNCEH',
}

In [31]:
# Dictionary of collections : UO
dict_Collections = scrap_collection_tree(repoLink, dict_UO, university, l_UO, DF_MSTheses_dir)